# Resolução de Sistemas Não-Lineares

### Imports

In [1]:
from numpy.random import randn
from numpy import ndarray, arange, fromiter, errstate
from typing import Callable
from itertools import count
from math import e

### Cálculo da Matriz Jacobiana

In [2]:
def secante(func: Callable, x: ndarray, e = 1e-15):

  def diff(i: int):
    b = x.copy(); b[i] += e
    fx, fb = func(*x), func(*b)
    return (fb - fx) / (b[i] - x[i])

  n = x.size
  return fromiter((diff(i) for i in arange(n)), float, n)

### Método de Gauss-Jacobi

In [3]:
def gauss_jacobi(A: ndarray, b: ndarray, e = 1e-15):
  n = A.shape[0]
  diag = A.diagonal()

  x = randn(n)
  C = -A / diag.reshape(n, 1)
  C.flat[::n+1] = 0
  G = b / diag

  cont = count()
  while next(cont) < 100:
    x_ant, x = x, C @ x + G
    if max(abs(x - x_ant)) < e: return x

  raise FloatingPointError

### Método de Newton

In [4]:
def newton(A: tuple[Callable], e = 1e-15):
  ERRO1 = "Qtd. diferente de variáveis nas funções!"
  ERRO2 = "Qtd. de variáveis difere da de funções!"

  try: [n] = set(func.__code__.co_argcount for func in A)
  except ValueError: raise ValueError(ERRO1) from None
  if n != len(A): raise ValueError(ERRO2)

  @errstate(all = "raise")
  def newton(x: ndarray):
    while 1:
      F = -fromiter((func(*x) for func in A), float, n)
      J = fromiter((secante(func, x) for func in A), f"{n}f8", n)
      x += (s := gauss_jacobi(J, F))
      if max(abs(s)) < e: break

  while 1:
    try: newton(x := randn(n))
    except FloatingPointError: continue
    else: break

  return x, fromiter((func(*x) for func in A), float, n)

### Execução do código

In [7]:
A = (
  lambda x, y: x**2 + y**2 - 2,
  lambda x, y: e**(x-1) + y**3 - 2
)

newton(A)

(array([-0.71374741,  1.22088682]), array([0., 0.]))